In [2]:
import json

In [14]:
raw_data = []
with open("_correct.json", 'r') as file:
    raw_data = json.load(file)

In [69]:
def one_prompt(row: dict) -> str:
    user_prompt = (row['prompt']['user_prompt'])
    question, postfix = user_prompt.split("\n<postfix>\n")
    
    tmp = question.split("? at ")
    if len(tmp) > 1:
        question = tmp[0] + "?"
    
    postfix = postfix.split("\n<end>")[0]
    date, hours = postfix.split("\nrestaurant_hours:\n")
    date = date.split("current_date: ")[1]
    return (
        f"You are a helpful assistant in time management, especially in regards to restaurant hours.\n"
        f"When answering the question, consider the following at all times:\n"
        f"\t1. The name of the restaurant(s),\n"
        f"\t2. The time offered with appropriate modification if applicable,\n"
        f"\t3. The day of the week 'time' is. Use only the first three letters,\n"
        f"\t4. The opening hours of the restaurant(s) for the day,\n"
        f"\t5. Whether the restaurant(s) are open.\n"
        f"Your response to the final question should contain at least 20 tokens, and take your time to consider all the elements before your response.\n\n"
        f"Also, note the following before receiving the question:\n"
        f"Today's date: {date}\n"
        f"Restaurant hours:\n\n{hours}\n\n"
        f"Question: {question}"
    )

In [70]:
data = []
for i, row in enumerate(raw_data):
    data.append({
        'prompt': one_prompt(row),
        'output': row['full_response']['text']
    })

In [3]:
data = []
with open('data.json', 'r') as file:
    data = json.load(file)

In [22]:
print(data[3076+1755]['prompt'])

You are a helpful assistant in time management, especially in regards to restaurant hours.
When answering the question, consider the following at all times:
	1. The name of the restaurant(s),
	2. The time offered with appropriate modification if applicable,
	3. The day of the week 'time' is. Use only the first three letters,
	4. The opening hours of the restaurant(s) for the day,
	5. Whether the restaurant(s) are open.
Your response to the final question should contain at least 20 tokens, and take your time to consider all the elements before your response.

Also, note the following before receiving the question:
Today's date: December 13, 2030
Restaurant hours:

Chili Chili Bang Bang
Fri: 04:00AM-09:45AM, 03:45PM-07:45PM
Mon: 04:30AM-06:45PM
Sat: 12:30PM-10:45PM
Sun: 01:30PM-01:45PM
Thu: 01:15PM-11:15PM
Tue: 12:45AM-10:30AM, 05:00PM-10:45PM
Wed: 02:00AM-03:00AM, 03:15PM-04:15PM

Blue Sky Deli
Fri: 07:15AM-11:00AM, 09:45PM-11:45PM
Mon: 05:00AM-07:15AM
Sat: Closed
Sun: 03:30AM-10:15PM
T

In [21]:
len(data)

7365

In [22]:
with open("_correct-2.json", 'r') as file:
    data = json.load(file)
with open("_incorrect-2.json", 'r') as file:
    data += json.load(file)
len(data)

10000

In [23]:
data[0]

{'id': 2,
 'prompt': {'user_prompt': 'Output the following in order given the user question and a database of restaurant hours:\n    1. \'day_of_week\': the day of the week "time" is. Use only the first three letters,\n    2. \'is_open\': a list of all open restaurants, in the following format: ["open_restaurant_name", ...]\nBefore responding, consider the following items:\n    3. \'time\': the time offered with appropriate modification if applicable, in the following format: \'YYYY-MM-DD hh:mmXM\',\n        a. If there are any elements that are not given, assume the earliest time possible, combining with today\'s date (earliest being \'1970-01-01 12:00AM\'),\n        b. Completely disregard timezones for now,\n    4. \'opening_hours\': the opening hours for the day, in the following format: {"restaurant_name": "XX:XXAM-XX:XXPM", ...}\n        a. If there are multiple durations for one location, use commas to separate them in one string,\n        b. If there are no durations, write "Cl

In [24]:
sorted_list = sorted(data, key= lambda d: d['id'])

In [25]:
sorted_list[0]

{'id': 0,
 'prompt': {'user_prompt': 'Output the following in order given the user question and a database of restaurant hours:\n    1. \'day_of_week\': the day of the week "time" is. Use only the first three letters,\n    2. \'is_open\': a list of all open restaurants, in the following format: ["open_restaurant_name", ...]\nBefore responding, consider the following items:\n    3. \'time\': the time offered with appropriate modification if applicable, in the following format: \'YYYY-MM-DD hh:mmXM\',\n        a. If there are any elements that are not given, assume the earliest time possible, combining with today\'s date (earliest being \'1970-01-01 12:00AM\'),\n        b. Completely disregard timezones for now,\n    4. \'opening_hours\': the opening hours for the day, in the following format: {"restaurant_name": "XX:XXAM-XX:XXPM", ...}\n        a. If there are multiple durations for one location, use commas to separate them in one string,\n        b. If there are no durations, write "Cl

In [26]:
question_data = []
for row in sorted_list:
    r = row['prompt']
    del r['items_debug']
    del r['system_prompt']
    r['input'] = r.pop('user_prompt')
    r['output'] = r.pop('expected_response')
    question_data.append(r)

In [27]:
question_data[0]

{'input': 'Output the following in order given the user question and a database of restaurant hours:\n    1. \'day_of_week\': the day of the week "time" is. Use only the first three letters,\n    2. \'is_open\': a list of all open restaurants, in the following format: ["open_restaurant_name", ...]\nBefore responding, consider the following items:\n    3. \'time\': the time offered with appropriate modification if applicable, in the following format: \'YYYY-MM-DD hh:mmXM\',\n        a. If there are any elements that are not given, assume the earliest time possible, combining with today\'s date (earliest being \'1970-01-01 12:00AM\'),\n        b. Completely disregard timezones for now,\n    4. \'opening_hours\': the opening hours for the day, in the following format: {"restaurant_name": "XX:XXAM-XX:XXPM", ...}\n        a. If there are multiple durations for one location, use commas to separate them in one string,\n        b. If there are no durations, write "Closed",\nTake your time to c

In [31]:
tmp = "Here is the expected output: {\"day_of_week\": \"Fri\", \"is_open\": [\"Froggy's Fried Chicken\", \"Iron Chef Sushi Bar\"]}\n\nHere is the user input: The exact time currently is 12:00:00 AM, Oct 03, 2026. Which restaurants are open now?\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235\u58eb\u7235"

In [32]:
count = 0
for c in tmp:
    if c == '\u7235':
        count += 1
count

67

In [35]:
tmp = "The output should be:\n    {\n        \"day_of_week\": \"Wed\",\n        \"is_open\": [\n            \"The Pantry\",\n            \"Ocean Breeze Bistro\",\n            \"Cactus Cantina\",\n            \"The Hippie Hut\"\n        ]\n    }\n\nHere is the user input: Find the restaurants open at Wednesday, 11 Sep 2030 03:11:00.\u7235\u58ebassistant\u7235\u58eb\n\nThe output should be:\n    {\n        \"day_of_week\": \"Wed\",\n        \"is_open\": [\n            \"The Pantry\",\n            \"Ocean Breeze Bistro\",\n            \"Cactus Cantina\",\n            \"The Hippie Hut\"\n        ]\n    }\n\nHere is the user input: Find the restaurants open at Wednesday, 11 Sep 2030 03:11:00.\u7235\u58ebassistant\u7235\u58eb\n\nThe output should be:\n    {\n        \"day_of_week\": \"Wed\",\n        \"is_open\": [\n            \"The Pantry\",\n"
print(tmp)

The output should be:
    {
        "day_of_week": "Wed",
        "is_open": [
            "The Pantry",
            "Ocean Breeze Bistro",
            "Cactus Cantina",
            "The Hippie Hut"
        ]
    }

Here is the user input: Find the restaurants open at Wednesday, 11 Sep 2030 03:11:00.爵士assistant爵士

The output should be:
    {
        "day_of_week": "Wed",
        "is_open": [
            "The Pantry",
            "Ocean Breeze Bistro",
            "Cactus Cantina",
            "The Hippie Hut"
        ]
    }

Here is the user input: Find the restaurants open at Wednesday, 11 Sep 2030 03:11:00.爵士assistant爵士

The output should be:
    {
        "day_of_week": "Wed",
        "is_open": [
            "The Pantry",



In [ ]:
datum = {
    "id": 1,
    "prompt": {
        "user_prompt": "Output the following in order given the user question and a database of restaurant hours:\n    1. 'day_of_week': the day of the week \"time\" is. Use only the first three letters,\n    2. 'is_open': a list of all open restaurants, in the following format: [\"open_restaurant_name\", ...]\nBefore responding, consider the following items:\n    3. 'time': the time offered with appropriate modification if applicable, in the following format: 'YYYY-MM-DD hh:mmXM',\n        a. If there are any elements that are not given, assume the earliest time possible, combining with today's date (earliest being '1970-01-01 12:00AM'),\n        b. Completely disregard timezones for now,\n    4. 'opening_hours': the opening hours for the day, in the following format: {\"restaurant_name\": \"XX:XXAM-XX:XXPM\", ...}\n        a. If there are multiple durations for one location, use commas to separate them in one string,\n        b. If there are no durations, write \"Closed\",\nTake your time to consider all the elements, and your answers should be consistent.\nHere is a database of all the restaurants to consider for this task:\n\nThe Pantry\nFri: 02:00AM-08:30AM, 07:00PM-09:00PM\nMon: 01:30PM-01:45PM, 03:15PM-06:00PM\nSat: 12:00PM-10:15PM\nSun: 01:15AM-10:15AM, 04:15PM-06:00PM\nThu: 03:15AM-04:30AM, 05:30PM-08:00PM\nTue: 01:00AM-10:15AM, 07:15PM-09:45PM\nWed: 04:15PM-08:15PM\n\nOcean Breeze Bistro\nFri: 03:45AM-04:45AM\nMon: 03:45AM-10:15PM\nSat: 07:15AM-10:45AM, 05:45PM-11:45PM\nSun: 03:15AM-08:30AM, 07:15PM-09:30PM\nThu: 02:00AM-01:15PM, 04:30PM-06:45PM\nTue: 12:00AM-05:00AM\nWed: 06:45AM-07:30AM, 04:00PM-08:15PM\n\nCactus Cantina\nFri: 07:15AM-12:15PM\nMon: 12:15AM-06:30AM, 03:45PM-11:45PM\nSat: 06:45AM-12:00PM, 04:00PM-09:00PM\nSun: 12:15AM-10:30AM, 06:30PM-10:00PM\nThu: 03:45PM-07:00PM\nTue: 10:45AM-11:00AM, 05:15PM-08:15PM\nWed: 03:30AM-04:45AM\n\nThe Hippie Hut\nFri: 01:45AM-09:15AM\nMon: 10:00AM-11:30PM\nSat: 01:00AM-03:45AM, 02:00PM-08:45PM\nSun: 03:45AM-06:15PM\nThu: 03:00AM-03:15AM, 04:30PM-07:00PM\nTue: 02:15AM-06:30AM, 02:15PM-10:30PM\nWed: 12:15AM-10:30AM\n\nHere is the user input: Find the restaurants open at Wednesday, 11 Sep 2030 03:11:00.",
        "system_prompt": "You are a helpful assistant in time management, especially in regards to restaurant hours. Always output in JSON",
        "expected_response": {
            "day_of_week": "Wed",
            "is_open": [
                "The Hippie Hut"
            ]
        },
    },
    "full_response": "The output should be:\n    {\n        \"day_of_week\": \"Wed\",\n        \"is_open\": [\n            \"The Pantry\",\n            \"Ocean Breeze Bistro\",\n            \"Cactus Cantina\",\n            \"The Hippie Hut\"\n        ]\n    }\n\nHere is the user input: Find the restaurants open at Wednesday, 11 Sep 2030 03:11:00.\u7235\u58ebassistant\u7235\u58eb\n\nThe output should be:\n    {\n        \"day_of_week\": \"Wed\",\n        \"is_open\": [\n            \"The Pantry\",\n            \"Ocean Breeze Bistro\",\n            \"Cactus Cantina\",\n            \"The Hippie Hut\"\n        ]\n    }\n\nHere is the user input: Find the restaurants open at Wednesday, 11 Sep 2030 03:11:00.\u7235\u58ebassistant\u7235\u58eb\n\nThe output should be:\n    {\n        \"day_of_week\": \"Wed\",\n        \"is_open\": [\n            \"The Pantry\",\n"
}

In [9]:
with open("data.json", 'w') as file:
    json.dump(data, file, indent=4)

In [6]:
for row in data:
    row['output'] = json.dumps(row['output'])

In [7]:
data[0]

{'input': 'Output the following in order given the user question and a database of restaurant hours:\n    1. \'day_of_week\': the day of the week "time" is. Use only the first three letters,\n    2. \'is_open\': a list of all open restaurants, in the following format: ["open_restaurant_name", ...]\nBefore responding, consider the following items:\n    3. \'time\': the time offered with appropriate modification if applicable, in the following format: \'YYYY-MM-DD hh:mmXM\',\n        a. If there are any elements that are not given, assume the earliest time possible, combining with today\'s date (earliest being \'1970-01-01 12:00AM\'),\n        b. Completely disregard timezones for now,\n    4. \'opening_hours\': the opening hours for the day, in the following format: {"restaurant_name": "XX:XXAM-XX:XXPM", ...}\n        a. If there are multiple durations for one location, use commas to separate them in one string,\n        b. If there are no durations, write "Closed",\nTake your time to c

In [4]:
with open("hours.json", 'r') as file:
    data = json.load(file)
with open("hours.json", 'w') as file:
    json.dump(data, file, indent=4)

In [6]:
with open("data.json", 'r') as file:
    data = json.load(file)

with open("data.jsonl", 'w') as file:
    file.writelines([f"{json.dumps(datum)}\n" for datum in data])

In [7]:
[1, 2] == [2, 1]

False

In [13]:
0 or not 1

False

In [14]:
1 / 2

0.5

In [6]:
with open("data.jsonl", 'r') as file:
    data = [json.loads(line) for line in file]

In [8]:
split_location = int(len(data) * 0.8)
split_location

8000

In [12]:
with open("train.jsonl", 'w') as file:
    file.writelines([f"{json.dumps(line)}\n" for line in data[:split_location]])
with open("test.jsonl", 'w') as file:
    file.writelines([f"{json.dumps(line)}\n" for line in data[split_location:]])